In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from w2vVectorizer import text2vec
import numpy as np

from sklearn.preprocessing import LabelBinarizer

import sys
sys.path.append('../..')

import utils.dbUtils
import utils.gensimUtils

torch.manual_seed(1)

In [2]:
vectorizer = text2vec()

Loading word2vec model...
Model Loaded


In [5]:
train = utils.dbUtils.TokenizedIterator('news_cleaned', filters = {'type' : {'$in' : ['fake', 'reliable']}, 'domain' : {'$nin' : ['nytimes.com', 'beforeitsnews.com']}}, limit = 10)
y_train = np.array([x for x in train.iterTags()])

In [67]:
encoder = LabelBinarizer(['fake', 'reliable'])

TypeError: '>=' not supported between instances of 'list' and 'int'

In [14]:
lb = LabelBinarizer()
lb.fit(y_train)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [17]:
y_train = lb.transform(y_train).flatten()

In [68]:
y_train

array([1, 0, 0, 1, 1, 1, 1, 1, 1, 1])

In [62]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, x):
        print(x)
        lstm_out, _ = self.lstm(x)
        tag_space = self.hidden2tag(lstm_out.view(len(x), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [57]:
t = torch.tensor(vectorizer.transform(train[0]), dtype=torch.float32).reshape(1, -1, 300)
#model(torch.tensor([vectorizer.transform(train[0]), vectorizer.transform(train[1])]))

In [58]:
t.shape

torch.Size([1, 201, 300])

In [65]:
model = LSTMTagger(300, 300, 1)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [66]:
with torch.no_grad():
    t = torch.tensor(vectorizer.transform(train[0]), dtype=torch.float32).reshape(1, -1, 300)
    tag_scores = model(t)
    print(tag_scores)

tensor([[[ 0.2207, -0.2256,  0.0325,  ..., -0.3359, -0.1475, -0.3223],
         [ 0.4235,  0.2254, -1.2741,  ..., -0.2456,  1.7554,  0.2808],
         [ 0.0206, -0.1157, -0.4102,  ..., -0.2773,  0.0024, -0.1992],
         ...,
         [-0.0231, -0.0156, -0.0840,  ..., -0.0498, -0.0126, -0.3125],
         [ 0.2266,  0.0564, -0.0962,  ..., -0.0300, -0.0082, -0.2109],
         [ 0.1553,  0.0942, -0.1934,  ..., -0.1030,  0.0603,  0.1426]]])


RuntimeError: size mismatch, m1: [1 x 60300], m2: [300 x 1] at /opt/conda/conda-bld/pytorch_1556653114079/work/aten/src/TH/generic/THTensorMath.cpp:961